---
## CSE 482 - Tweet Loading and Processing
### Jack Nugent
Load Tweets related to Elon Musk and prepare the text for sentiment analysis.

---

In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
import json
import pandas as pd
import re

In [2]:
#!pip install emoji
import emoji

In [3]:
# Replace with Your Keys
C_KEY = None
C_SECRET = None
A_TOKEN_KEY = None
A_TOKEN_SECRET = None
BEARER = None

In [4]:
# Jack Nugent Keys
C_KEY = 'qDLzDR1kgKZrwc0nfCiknCNak'
C_SECRET = '0siNqGf4CSQVM2EL2JTYKq0fqBN7VXtTXv2MSyUAiiX5V1DBeG'
A_TOKEN_KEY = '762757536827121664-YNdvEO2FsulfLcjUJcezb5OMRx3Nlgs'
A_TOKEN_SECRET = '88DYUdSwCfWmNlBWbY9It8uFqajxE7NMMRS4moZbdjz4k'
BEARER = "AAAAAAAAAAAAAAAAAAAAAOCnhQEAAAAABJxPmDWLGJXfFdBa9Nprs%2BfFHus%3De6Qt9gjlzObpDy4HIWJWMQUXXLE6nkNUF3Drkv6QLkZezBn4JU"

---
### Loading Tweets

In [5]:
data_json = []

In [6]:
files = ['output_geotag_day1.json']

for file in files:
    f = open(file)

    data_json = []
    for line in f:
        try:
            data_json.append(json.loads(line))
        # Discard Tweets with invalid chars
        except:
            pass

    f.close()

In [24]:
df = pd.DataFrame(columns=['Text', 'Place_id', 'Tweet_id', 'Date', 'Time'])

for tweet in data_json:
    date, time = tweet['data']['created_at'].split('T')
    # Handles Retweets that somehow slip through filter
    if 'RT @' not in tweet['data']['text']:
        df.loc[len(df.index)] = [tweet['data']['text'], tweet['data']['geo']['place_id'], tweet['data']['id'], date, time[:-5]]

print("Number of Tweets:",len(df))
df

Number of Tweets: 312


,Text,Place_id,Tweet_id,Date,Time
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23
...,...,...,...,...,...
307,@LH44Tobeyy @elonmusk Later bro,000a381c49091836,1595512662460403725,2022-11-23,20:20:36
308,Cannot be emphasized enough how huge a loser E...,10de09f288b1665c,1595512757746511872,2022-11-23,20:20:59
309,@teslaownersSV @elonmusk Plus she just got int...,0e45c1c6e13dff2a,1595512809198325760,2022-11-23,20:21:11
310,@DileepM8485 @ACTBrigitte I think you're a Mus...,c95cdb2a983262e5,1595512859722936320,2022-11-23,20:21:23


---
### Processing Text


#### Cleaning Text

In [30]:
%%time
df['Words'] = df['Text'].str.split()
df['Hashtags'] = None
for x in range(len(df)):
    # Handle '@' Mentions
    df['Words'][x] = [i for i in df['Words'][x] if not re.compile('@').match(i)]
    
    # Handle links
    df['Words'][x] = [i for i in df['Words'][x] if not re.compile('http').match(i)]
    
    # Hashtags
    df['Hashtags'][x] = [word for word in df['Words'][x] if '#' in word]
    df['Words'][x] = [word for word in df['Words'][x] if word not in df['Hashtags'][x]]
    
    # Handle Emojis and Punctuation
    words = []
    for word in df.iloc[x]['Words']:
        for c in word:
            if c in emoji.EMOJI_DATA or c in '''!()-[]{};:'"\,<>./?@#$%^&*_~''':
                word = word.replace(c, '')
        # Lowercase Words
        words.append(word.lower())
    df['Words'][x] = words
    
    # Remove Empty Values
    for word in df['Words'][x]:
        if word == '':
            df['Words'][x].remove(word)
            

CPU times: user 312 ms, sys: 6.91 ms, total: 319 ms
Wall time: 313 ms


In [31]:
df

,Text,Place_id,Tweet_id,Date,Time,Words,Hashtags
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, do, you, approve, or, di...",[]
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51,"[twitter, and, elon, has, the, same, vibes, wh...",[]
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08,"[then, why, do, i, see, him, so, frequently, o...",[]
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18,"[this, is, hysterical, thanks, for, laugh, whi...",[]
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23,"[elon, musk, can, show, us, the, way, but, it’...",[]
...,...,...,...,...,...,...,...
307,@LH44Tobeyy @elonmusk Later bro,000a381c49091836,1595512662460403725,2022-11-23,20:20:36,"[later, bro]",[]
308,Cannot be emphasized enough how huge a loser E...,10de09f288b1665c,1595512757746511872,2022-11-23,20:20:59,"[cannot, be, emphasized, enough, how, huge, a,...",[]
309,@teslaownersSV @elonmusk Plus she just got int...,0e45c1c6e13dff2a,1595512809198325760,2022-11-23,20:21:11,"[plus, she, just, got, interviewed, by, katy, ...",[]
310,@DileepM8485 @ACTBrigitte I think you're a Mus...,c95cdb2a983262e5,1595512859722936320,2022-11-23,20:21:23,"[i, think, youre, a, musk, worship, bot]",[]


---
### NLTK Cleaning

In [32]:
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package wordnet to /home/nugentj3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nugentj3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nugentj3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nugentj3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


---
#### Stopword Removal

In [33]:
stop_words = set(stopwords.words('english'))

In [34]:
# Stopword Removal
for x in range(len(df)):
    df['Words'][x] = [word for word in df['Words'][x] if word not in stop_words]
df

,Text,Place_id,Tweet_id,Date,Time,Words,Hashtags
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, approve, disapprove, elo...",[]
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51,"[twitter, elon, vibes, trump, got, elected, la...",[]
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08,"[see, frequently, twitter, stop, posting]",[]
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18,"[hysterical, thanks, laugh, prep, thanksgiving...",[]
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23,"[elon, musk, show, us, way, it’s, hard, work, ...",[]
...,...,...,...,...,...,...,...
307,@LH44Tobeyy @elonmusk Later bro,000a381c49091836,1595512662460403725,2022-11-23,20:20:36,"[later, bro]",[]
308,Cannot be emphasized enough how huge a loser E...,10de09f288b1665c,1595512757746511872,2022-11-23,20:20:59,"[cannot, emphasized, enough, huge, loser, elon...",[]
309,@teslaownersSV @elonmusk Plus she just got int...,0e45c1c6e13dff2a,1595512809198325760,2022-11-23,20:21:11,"[plus, got, interviewed, katy, tur, msnbc, sur...",[]
310,@DileepM8485 @ACTBrigitte I think you're a Mus...,c95cdb2a983262e5,1595512859722936320,2022-11-23,20:21:23,"[think, youre, musk, worship, bot]",[]


---
#### Stemming

In [35]:
df['Stemmed'] = None
for x in range(len(df)):
    df['Stemmed'][x] = [SnowballStemmer(language='english').stem(word) for word in df['Words'][x]]
#df

---
#### Lemmatization

In [36]:
df['Lemmed'] = None
for x in range(len(df)):
    df['Lemmed'][x] = [WordNetLemmatizer().lemmatize(word) for word in df['Words'][x]]
#df

---
#### Part of Speech Tagging

In [37]:
#POS Tagging
df['POS_Tags'] = None
for x in range(len(df)):
    df['POS_Tags'][x] = nltk.pos_tag(df['Lemmed'][x])
df

,Text,Place_id,Tweet_id,Date,Time,Words,Hashtags,Stemmed,Lemmed,POS_Tags
0,.@EconUS/@YouGovAmerica:\n\nDo you approve or ...,6079430106fe2a39,1595482958764425218,2022-11-23,18:22:34,"[econusyougovamerica, approve, disapprove, elo...",[],"[econusyougovamerica, approv, disapprov, elon,...","[econusyougovamerica, approve, disapprove, elo...","[(econusyougovamerica, RB), (approve, VB), (di..."
1,Twitter and Elon has the same vibes when Trump...,94965b2c45386f87,1595483029048369152,2022-11-23,18:22:51,"[twitter, elon, vibes, trump, got, elected, la...",[],"[twitter, elon, vibe, trump, got, elect, laugh...","[twitter, elon, vibe, trump, got, elected, lau...","[(twitter, NN), (elon, NN), (vibe, NN), (trump..."
2,@RonFilipkowski Then why do I see him so frequ...,2d83c71ce16cd187,1595483099735154688,2022-11-23,18:23:08,"[see, frequently, twitter, stop, posting]",[],"[see, frequent, twitter, stop, post]","[see, frequently, twitter, stop, posting]","[(see, VB), (frequently, RB), (twitter, JJR), ..."
3,@timothy_wall @elonmusk This is hysterical. Th...,018244f41aa15ac3,1595483141879353344,2022-11-23,18:23:18,"[hysterical, thanks, laugh, prep, thanksgiving...",[],"[hyster, thank, laugh, prep, thanksgiv, need]","[hysterical, thanks, laugh, prep, thanksgiving...","[(hysterical, JJ), (thanks, NNS), (laugh, IN),..."
4,Elon Musk can show us the way but it’s we who ...,011add077f4d2da3,1595483162289016832,2022-11-23,18:23:23,"[elon, musk, show, us, way, it’s, hard, work, ...",[],"[elon, musk, show, us, way, it, hard, work, ma...","[elon, musk, show, u, way, it’s, hard, work, m...","[(elon, NN), (musk, NN), (show, NN), (u, JJ), ..."
...,...,...,...,...,...,...,...,...,...,...
307,@LH44Tobeyy @elonmusk Later bro,000a381c49091836,1595512662460403725,2022-11-23,20:20:36,"[later, bro]",[],"[later, bro]","[later, bro]","[(later, RB), (bro, NN)]"
308,Cannot be emphasized enough how huge a loser E...,10de09f288b1665c,1595512757746511872,2022-11-23,20:20:59,"[cannot, emphasized, enough, huge, loser, elon...",[],"[cannot, emphas, enough, huge, loser, elon, mu...","[cannot, emphasized, enough, huge, loser, elon...","[(cannot, NN), (emphasized, VBD), (enough, RB)..."
309,@teslaownersSV @elonmusk Plus she just got int...,0e45c1c6e13dff2a,1595512809198325760,2022-11-23,20:21:11,"[plus, got, interviewed, katy, tur, msnbc, sur...",[],"[plus, got, interview, kati, tur, msnbc, surpr...","[plus, got, interviewed, katy, tur, msnbc, sur...","[(plus, CC), (got, VBD), (interviewed, VBN), (..."
310,@DileepM8485 @ACTBrigitte I think you're a Mus...,c95cdb2a983262e5,1595512859722936320,2022-11-23,20:21:23,"[think, youre, musk, worship, bot]",[],"[think, your, musk, worship, bot]","[think, youre, musk, worship, bot]","[(think, VB), (youre, NN), (musk, FW), (worshi..."
